### There are cases when you have to parse data before loading into DataFrame
* Data is not of csv format.
* Further parsing of data elements is required
* ...


#### Typical workflow
1. read data as an RDD of text lines.
2. split each line into a list of elements.
3. parse each element (if needed)
4. convert each line into a **_sqlcontext Row_**, i.e. RDD of **_Rows_**
5. convert the RDD into a DataFrame


##### *Row* is a list of key/value pairs represented as
$ [ col_1=value_1, col_2=value_2, ..., col_n = value_n ]$

#### Example: Analyzing Web log data

In [1]:
from pyspark.sql import Row
log_file = sc.textFile("./data/log_file.txt")

In [2]:
import shlex
splits = log_file.map(lambda row: shlex.split(row))
splits.takeSample(True, 5)

[['butterfly.europa.com', '[30:00:41:57]', 'GET /enviro/gif/book.gif HTTP/1.0', '304', '0'], ['ix-eve-wa2-02.ix.netcom.com', '[30:01:35:38]', 'GET /icons/book.gif HTTP/1.0', '200', '156'], ['204.188.47.212', '[30:00:53:15]', 'GET /EPA-WATER/1995/August/Day-29/ HTTP/1.0', '200', '799'], ['reggae.iinet.net.au', '[30:04:30:13]', 'GET /logos/small_ftp.gif HTTP/1.0', '304', '0'], ['ix-pen-nj1-13.ix.netcom.com', '[30:00:37:25]', 'GET /./enviro/gif/banner.gif HTTP/1.0', '200', '14887']]

In [3]:
def create_schema(row):
  ip = row[0]
  date = row[1].replace('[', '').replace(']', '')
  tokens = row[2].split(' ')
  protocol = tokens[0]
  url = tokens[1].split('?')[0]
  status = row[3]
  time = None if row[4] == '-' else int(row[4]) 
  return Row(ip=ip, date=date, protocol=protocol, url=url, status=status, time=time)

In [11]:
row_data = splits.map(create_schema)


from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

row_data.take(10)

[Row(date='29:23:53:25', ip='141.243.1.172', protocol='GET', status='200', time=1497, url='/Software.html'), Row(date='29:23:53:36', ip='query2.lycos.cs.cmu.edu', protocol='GET', status='200', time=1325, url='/Consumer.html'), Row(date='29:23:53:53', ip='tanuki.twics.com', protocol='GET', status='200', time=1014, url='/News.html'), Row(date='29:23:54:15', ip='wpbfl2-45.gate.net', protocol='GET', status='200', time=4889, url='/'), Row(date='29:23:54:16', ip='wpbfl2-45.gate.net', protocol='GET', status='200', time=2624, url='/icons/circle_logo_small.gif'), Row(date='29:23:54:18', ip='wpbfl2-45.gate.net', protocol='GET', status='200', time=935, url='/logos/small_gopher.gif'), Row(date='29:23:54:19', ip='140.112.68.165', protocol='GET', status='200', time=2788, url='/logos/us-flag.gif'), Row(date='29:23:54:19', ip='wpbfl2-45.gate.net', protocol='GET', status='200', time=124, url='/logos/small_ftp.gif'), Row(date='29:23:54:19', ip='wpbfl2-45.gate.net', protocol='GET', status='200', time=156

In [5]:
schema_DF = sqlContext.createDataFrame(row_data)
schema_DF.createOrReplaceTempView('logs')
sample = sqlContext.sql('SELECT * FROM logs LIMIT 10').collect()   # returns list
for row in sample:
    print row

Row(date=u'29:23:53:25', ip=u'141.243.1.172', protocol=u'GET', status=u'200', time=1497, url=u'/Software.html')
Row(date=u'29:23:53:36', ip=u'query2.lycos.cs.cmu.edu', protocol=u'GET', status=u'200', time=1325, url=u'/Consumer.html')
Row(date=u'29:23:53:53', ip=u'tanuki.twics.com', protocol=u'GET', status=u'200', time=1014, url=u'/News.html')
Row(date=u'29:23:54:15', ip=u'wpbfl2-45.gate.net', protocol=u'GET', status=u'200', time=4889, url=u'/')
Row(date=u'29:23:54:16', ip=u'wpbfl2-45.gate.net', protocol=u'GET', status=u'200', time=2624, url=u'/icons/circle_logo_small.gif')
Row(date=u'29:23:54:18', ip=u'wpbfl2-45.gate.net', protocol=u'GET', status=u'200', time=935, url=u'/logos/small_gopher.gif')
Row(date=u'29:23:54:19', ip=u'140.112.68.165', protocol=u'GET', status=u'200', time=2788, url=u'/logos/us-flag.gif')
Row(date=u'29:23:54:19', ip=u'wpbfl2-45.gate.net', protocol=u'GET', status=u'200', time=124, url=u'/logos/small_ftp.gif')
Row(date=u'29:23:54:19', ip=u'wpbfl2-45.gate.net', proto

In [6]:
schema_DF.show()

+-----------+--------------------+--------+------+-----+--------------------+
|       date|                  ip|protocol|status| time|                 url|
+-----------+--------------------+--------+------+-----+--------------------+
|29:23:53:25|       141.243.1.172|     GET|   200| 1497|      /Software.html|
|29:23:53:36|query2.lycos.cs.c...|     GET|   200| 1325|      /Consumer.html|
|29:23:53:53|    tanuki.twics.com|     GET|   200| 1014|          /News.html|
|29:23:54:15|  wpbfl2-45.gate.net|     GET|   200| 4889|                   /|
|29:23:54:16|  wpbfl2-45.gate.net|     GET|   200| 2624|/icons/circle_log...|
|29:23:54:18|  wpbfl2-45.gate.net|     GET|   200|  935|/logos/small_goph...|
|29:23:54:19|      140.112.68.165|     GET|   200| 2788|  /logos/us-flag.gif|
|29:23:54:19|  wpbfl2-45.gate.net|     GET|   200|  124|/logos/small_ftp.gif|
|29:23:54:19|  wpbfl2-45.gate.net|     GET|   200|  156|     /icons/book.gif|
|29:23:54:19|  wpbfl2-45.gate.net|     GET|   200| 2788|  /logos

### Most visited URLs?

In [7]:
url_access = sqlContext.sql('''SELECT url, count(*) as counts FROM logs GROUP BY url
  ORDER BY counts DESC LIMIT 10''')#.collect()

url_access.show()
#for row in url_access:
#    print row
    


+--------------------+------+
|                 url|counts|
+--------------------+------+
|/icons/circle_log...|   128|
|                   /|   103|
|/logos/small_ftp.gif|    74|
|  /logos/us-flag.gif|    74|
|/logos/small_goph...|    71|
|     /icons/book.gif|    71|
|    /icons/ok2-0.gif|    69|
|/waisicons/unknow...|    62|
|   /cgi-bin/waisgate|    43|
|/cgi-bin/waisgate...|    26|
+--------------------+------+



### Most frequent visitors?

In [8]:
visitors = sqlContext.sql('''SELECT ip, count(*) as counts FROM logs GROUP BY ip
  ORDER BY counts DESC LIMIT 10''').collect()
for row in visitors:
    print row

Row(ip=u'ix-eve-wa2-02.ix.netcom.com', counts=126)
Row(ip=u'pm2-3.niia.net', counts=81)
Row(ip=u'202.32.50.6', counts=73)
Row(ip=u'bettong.client.uq.oz.au', counts=64)
Row(ip=u'macn352.riken.go.jp', counts=50)
Row(ip=u'sfsp03.slip.net', counts=47)
Row(ip=u'h46.s101.wcc.com', counts=46)
Row(ip=u'cragateway.cra.com.au', counts=44)
Row(ip=u'port11.annex1.naples.net', counts=43)
Row(ip=u'hmu4.cs.auckland.ac.nz', counts=39)


In [12]:
td = schema_DF.select("ip","time").filter(schema_DF.time > 3000).groupBy('ip').count().orderBy('count', ascending=False)#.show()
td.show()

+--------------------+-----+
|                  ip|count|
+--------------------+-----+
|hmu4.cs.auckland....|   19|
|      pm2-3.niia.net|   19|
|ix-eve-wa2-02.ix....|   18|
|     sfsp03.slip.net|   15|
|     203.251.228.110|   14|
|      168.95.125.161|   14|
|epsongw3.epson.co.jp|   13|
|  infoman.cisnet.com|   13|
|bettong.client.uq...|   12|
|       161.122.12.78|   11|
| macn352.riken.go.jp|   10|
|port11.annex1.nap...|   10|
|    h46.s101.wcc.com|    9|
|nnex02.ppp.uni-ma...|    9|
|daddylongleg.euro...|    9|
|cnts4p16.uwaterlo...|    9|
|        132.74.12.10|    8|
|  ext-ns.dpie.gov.au|    8|
|       scenic.mix.it|    7|
|         202.32.50.6|    7|
+--------------------+-----+
only showing top 20 rows



#### Can we change order of operations?

In [13]:
td = schema_DF.filter(schema_DF.time > 3000).select('ip','time').groupBy('ip').count().orderBy('count', ascending=False)
td.show()

+--------------------+-----+
|                  ip|count|
+--------------------+-----+
|      pm2-3.niia.net|   19|
|hmu4.cs.auckland....|   19|
|ix-eve-wa2-02.ix....|   18|
|     sfsp03.slip.net|   15|
|      168.95.125.161|   14|
|     203.251.228.110|   14|
|epsongw3.epson.co.jp|   13|
|  infoman.cisnet.com|   13|
|bettong.client.uq...|   12|
|       161.122.12.78|   11|
| macn352.riken.go.jp|   10|
|port11.annex1.nap...|   10|
|    h46.s101.wcc.com|    9|
|cnts4p16.uwaterlo...|    9|
|nnex02.ppp.uni-ma...|    9|
|daddylongleg.euro...|    9|
|  ext-ns.dpie.gov.au|    8|
|        132.74.12.10|    8|
|         202.32.50.6|    7|
|       scenic.mix.it|    7|
+--------------------+-----+
only showing top 20 rows



#### How to count visitors by _.com_, _.net_, etc?

In [15]:
tdrdd = td.rdd
tdrdd.take(10)

[Row(ip=u'pm2-3.niia.net', count=19), Row(ip=u'hmu4.cs.auckland.ac.nz', count=19), Row(ip=u'ix-eve-wa2-02.ix.netcom.com', count=18), Row(ip=u'sfsp03.slip.net', count=15), Row(ip=u'203.251.228.110', count=14), Row(ip=u'168.95.125.161', count=14), Row(ip=u'epsongw3.epson.co.jp', count=13), Row(ip=u'infoman.cisnet.com', count=13), Row(ip=u'bettong.client.uq.oz.au', count=12), Row(ip=u'161.122.12.78', count=11)]